In [1]:
import pandas as pd

In [2]:
family_file = pd.read_csv("..//data//2016//family_file.csv")
househld_file = pd.read_csv("..//data//2016//househld_file.csv")
person_file = pd.read_csv("..//data//2016//person_file.csv")
injury_epsd = pd.read_csv("..//data//2016//injury_epsd.csv")
adult_file = pd.read_csv("..//data//2016//adult_file.csv")
child_file = pd.read_csv("..//data//2016//child_file.csv")

In [3]:
import numpy as np

In [4]:
adult_file['FLUVACYR']=np.where(adult_file['FLUVACYR']==1,1,0)

In [5]:
adult_file=adult_file.fillna(99999)

In [6]:
from sklearn import cross_validation

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
adult_file=adult_file.drop(['FLUVACTP','FLUVAC_M','FLUVAC_Y'],axis=1)

In [8]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

data_x =  adult_file.drop(['FLUVACYR'],axis=1)
data_y = adult_file['FLUVACYR']
print(data_x.shape)
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(data_x, data_y)
model = SelectFromModel(lsvc, prefit=True)
data_x_new = model.transform(data_x)
print(data_x_new.shape)

(33028, 801)
(33028, 113)


In [9]:
selected_feature = pd.DataFrame(data_x.columns)

selected_feature.columns=['columns']

selected_feature['selection']=model.get_support()

selected_col_list = selected_feature[selected_feature['selection']==True]['columns'].tolist()

In [19]:
data_x = adult_file.drop(['FLUVACYR'],axis=1)# adult_file[selected_col_list]
data_y = adult_file['FLUVACYR']

X_train, X_test, y_train, y_test = cross_validation.train_test_split(data_x, data_y, test_size=0.6, random_state=42)

from sklearn import metrics
#from sklearn import tree
#clf = tree.DecisionTreeClassifier()

#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import SVC
#clf = OneVsRestClassifier(SVC(kernel="linear"))
#from sklearn.ensemble import RandomForestClassifier
#clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=10))
#from sklearn.naive_bayes import MultinomialNB
#clf = OneVsRestClassifier(MultinomialNB(alpha=0.01))
#from sklearn import svm
#clf = svm.SVC(kernel="linear")


from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)

#from sklearn import linear_model
#clf = linear_model.LogisticRegression(C=1e5)

clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print("Overall Accuracy = %s" %metrics.accuracy_score(y_pred,y_test))
print("Classification Matrix")
print(metrics.classification_report(y_pred,y_test))

Overall Accuracy = 0.71433617601
Classification Matrix
             precision    recall  f1-score   support

          0       0.79      0.72      0.75     11951
          1       0.63      0.70      0.66      7866

avg / total       0.72      0.71      0.72     19817



In [21]:
len(clf.feature_importances_)

801

In [22]:
feature_imp = pd.DataFrame(list(data_x.columns))

feature_imp.columns=['col_name']

feature_imp['imp_val']=clf.feature_importances_

feature_imp.sort_values(by=['imp_val'],ascending=False)

In [ ]:
svm_weights_selected = (clf.coef_ ** 2).sum(axis=0)
svm_weights_selected /= svm_weights_selected.max()

imp= pd.DataFrame(list(data_x.columns))

imp['imp']=svm_weights_selected

imp.sort_values(by=['imp'],ascending=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

imp.sort_values(by=['imp'],ascending=False)[0:50].plot.bar(x=0,y='imp')

In [38]:
new_data = adult_file[imp.sort_values(by=['imp'],ascending=False)[0:50][0].tolist()].copy()

In [41]:
new_data['FLUVACYR']=adult_file['FLUVACYR'].copy()

In [ ]:
data_x =  new_data.drop(['FLUVACYR'],axis=1)
data_y = new_data['FLUVACYR']

X_train, X_test, y_train, y_test = cross_validation.train_test_split(data_x, data_y, test_size=0.2, random_state=42)

from sklearn import svm
from sklearn import metrics
clf = svm.SVC(kernel="linear")


clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print("Overall Accuracy = %s" %metrics.accuracy_score(y_pred,y_test))
print("Classification Matrix")
print(metrics.classification_report(y_pred,y_test))

In [ ]:
svm_weights_selected = (clf.coef_ ** 2).sum(axis=0)
svm_weights_selected /= svm_weights_selected.max()

imp= pd.DataFrame(list(data_x.columns))

imp['imp']=svm_weights_selected

imp.sort_values(by=['imp'],ascending=False)